# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [39]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [40]:
weather = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Nikolskoye,59.70,30.79,42.01,86,90,11.18,RU,1604348317
1,Mar del Plata,-38.00,-57.56,66.20,55,90,12.75,AR,1604348317
2,Severo-Kuril'sk,50.68,156.12,43.30,82,100,23.13,RU,1604348317
3,Sinnamary,5.38,-52.95,78.62,86,100,14.56,GF,1604348317
4,Busselton,-33.65,115.33,55.99,84,78,11.23,AU,1604348318


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [64]:
figure_layout = {
    'width': '600px',
    'height': '550px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
lat_long = weather[['Lat','Long']]
heat = gmaps.heatmap_layer(lat_long, weights=weather['Humidity'])
fig.add_layer(heat)
fig

Figure(layout=FigureLayout(border='1px solid black', height='550px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [65]:
# Finding perfect fall weather
bestTemp = weather.loc[((weather['Max Temp'] >= 60) & (weather['Max Temp'] <= 70))]
lowHum = bestTemp.loc[bestTemp['Humidity'] <=20]
lowHum

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,Saint George,37.10,-113.58,68.0,20,1,4.70,US,1604348181
206,Rocky Mountain House,52.37,-114.92,66.2,16,93,12.75,CA,1604348393
249,Rupert,42.62,-113.68,66.2,20,1,8.05,US,1604348332


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [66]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key
}
# Empty lists for making the DF
names = []
lats = []
longs = []

# Get the hotel info
for index, row in lowHum.iterrows():
    params["location"] = str(row['Lat']) + "," + str(row['Long'])
    hotelData = requests.get(base_url, params=params).json()
    names.append(hotelData['results'][0]['name'])
    lats.append(hotelData['results'][0]['geometry']['location']['lat'])
    longs.append(hotelData['results'][0]['geometry']['location']['lng'])

# Build DF out of the hotel data
hotel_df = pd.DataFrame({
    'Hotel Name': names,
    'Lat': lats,
    'Lng': longs,
    'City': lowHum['City'],
    'Country': lowHum['Country']
})
hotel_df

,Hotel Name,Lat,Lng,City,Country
27,Tru By Hilton St. George,37.086557,-113.579159,Saint George,US
206,Best Western Rocky Mountain House Inn & Suites,52.360568,-114.908584,Rocky Mountain House,CA
249,Henry's at the Drift Inn,42.617360,-113.675214,Rupert,US


In [67]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [68]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='550px', margin='0 auto 0 auto', padding='1px', wi…